In [1]:
from pathlib import Path
import pandas as pd
import numpy as np


from sklearn.experimental import enable_halving_search_cv
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import TargetEncoder, OneHotEncoder, StandardScaler, LabelEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, HalvingGridSearchCV
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier
from sklearn import set_config
set_config(enable_metadata_routing=True)

from joblib import parallel_backend
from time import monotonic
from prince import FAMD, MCA

from utils.data_processing import load_data, raw_columns, full_dtypes, transform_datetime, df_ua_parser, transform_ipinfo, transform_packetinfo, transform_proxyinfo

In [2]:
data_path = Path("./data")
if not data_path.joinpath("first_ml_processing.csv").exists():
    # Must use clean_data function to load data 
    input_data_path = Path("./data/cybersecurity_attacks.csv")
    dtypes = {col: col_type for col, col_type in full_dtypes.items() if col in raw_columns}
    raw_data = load_data(input_data_path, dtype=dtypes)

    datetime_columns = ["Year", "Month", "Day", "Hour", "Minute", "Second", "DayOfWeek", "IsWeekend"]
    raw_data[datetime_columns] = transform_datetime(raw_data["Timestamp"])
    device_columns = ["String","Browser Name", "Browser Version", "Browser Minor", "Browser Patch",
                    "Browser Patch Minor", "OS Name", "OS Version", "OS Version Minor",
                    "OS Version Patch", "OS Version Patch Minor", "Device Brand", "Device Model",
                    "Device Type"]
    raw_data[device_columns] = df_ua_parser(raw_data["Device Information"])
    proxy_columns = ["Is Proxy"]
    raw_data[proxy_columns] = transform_proxyinfo(raw_data["Proxy Information"])
    ip_columns = ["Int Source IP", "Int Destination IP", "Global Source IP", "Global Destination IP"]
    raw_data[ip_columns] = transform_ipinfo(raw_data[["Source IP Address", "Destination IP Address"]])
    packet_columns = ["Packet Bin"]
    raw_data[packet_columns] = transform_packetinfo(raw_data["Packet Length"], scale=False)

    processed_data = raw_data.drop(columns=["Payload Data","Timestamp", "String", "Device Information",
                                    "Proxy Information", "Source IP Address", "Destination IP Address"])
    processed_data.to_csv(data_path.joinpath("first_ml_processing.csv"), index=False)
processed_data = pd.read_csv(data_path.joinpath("first_ml_processing.csv"))

In [3]:
Y_true = processed_data["Attack Type"].copy()
X_dataset = processed_data.copy().drop(columns=["Attack Type", "Browser Patch" , "Browser Patch Minor",
                                                "OS Version", "OS Version Minor", "OS Version Patch", "OS Version Patch Minor",
                                                "Device Type", "User Information", "Geo-location Data"])

# Feature Selection
## PCA Analysis

In [4]:
cat_cols = X_dataset.select_dtypes(include=["category","str"]).columns
num_cols = X_dataset.select_dtypes(include=["int64","float64"]).columns
bool_cols = X_dataset.select_dtypes(include=["bool"]).columns
passthrough_columns = [col for col in X_dataset.columns if col not in cat_cols and col not in bool_cols and col not in num_cols]
    
numeric_transformer = Pipeline(
        steps = [("scaler", StandardScaler())]
    )

categorical_transformer = Pipeline([
        ("imputer", SimpleImputer(missing_values=np.nan, strategy="constant", fill_value="unknown")),
        ("encoder", OrdinalEncoder())
        ])
boolean_transformer = Pipeline([
        ("encoder", TargetEncoder(target_type="binary")),
        ])

preprocessor = ColumnTransformer(
        transformers=[
            ("cat", categorical_transformer, cat_cols),
            ("num", numeric_transformer, num_cols),
            ("bool", boolean_transformer, bool_cols)
        ]
    )

param_grid = {
    "classifier__n_estimators": [100, 200],
    "classifier__max_depth": [None, 10, 20],
    "classifier__min_samples_split": [2, 10, 30],
    "pca__n_components": [2, 6, 10, 15]
}

pipeline = Pipeline(
        steps=[
            ("preprocessor", preprocessor),
            ("pca", PCA(random_state=124)),
            ("classifier", RandomForestClassifier(random_state=124))
        ])

relaunch =""
while relaunch.lower() not in ["y", "n"]:
    relaunch = input("Do you want to relaunch the grid search with raw data? (y/n) ")
if relaunch.lower() == "y":
    X_train, X_test, y_train, y_test = train_test_split(X_dataset, Y_true, test_size=0.2, stratify=Y_true, random_state=124)
    start_time = monotonic()
    gs = GridSearchCV(pipeline, param_grid, cv=5, n_jobs=1)
    gs.fit(X_train, y_train)
    print("Time taken to fit the model: %.2f seconds" % (monotonic() - start_time))
    print("Model score: %.3f" % gs.score(X_test, y_test))
    print("Best parameters: ", gs.best_params_)

In [5]:
cat_cols = X_dataset.select_dtypes(include=["category","str"]).columns
num_cols = X_dataset.select_dtypes(include=["int64","float64"]).columns
bool_cols = X_dataset.select_dtypes(include=["bool"]).columns
passthrough_columns = [col for col in X_dataset.columns if col not in cat_cols and col not in bool_cols and col not in num_cols]
    
numeric_transformer = Pipeline(
        steps = [("scaler", StandardScaler())]
    )

categorical_transformer = Pipeline([
        ("imputer", SimpleImputer(missing_values=np.nan, strategy="constant", fill_value="unknown")),
        ("encoder", OneHotEncoder(drop="first", handle_unknown="ignore"))
        ])
boolean_transformer = Pipeline([
        ("encoder", TargetEncoder(target_type="binary")),
        ])

preprocessor = ColumnTransformer(
        transformers=[
            ("cat", categorical_transformer, cat_cols),
            ("num", numeric_transformer, num_cols),
            ("bool", boolean_transformer, bool_cols)
        ]
    )

param_grid = {
    "classifier__n_estimators": [100, 200],
    "classifier__max_depth": [None, 10, 20],
    "classifier__min_samples_split": [2, 10, 30],
    "pca__n_components": [2, 6, 10, 15]
}

pipeline = Pipeline(
        steps=[
            ("preprocessor", preprocessor),
            ("pca", PCA(random_state=124)),
            ("classifier", RandomForestClassifier(random_state=124))
        ])

relaunch =""
while relaunch.lower() not in ["y", "n"]:
    relaunch = input("Do you want to relaunch the grid search with an ordinal encoder for categorical data? (y/n) ")
if relaunch.lower() == "y":
    X_train, X_test, y_train, y_test = train_test_split(X_dataset, Y_true, test_size=0.2, stratify=Y_true, random_state=124)
    start_time = monotonic()
    gs = GridSearchCV(pipeline, param_grid, cv=5)
    with parallel_backend('threading', n_jobs=2):
        gs.fit(X_train, y_train)
    print("Time taken to fit the model: %.2f seconds" % (monotonic() - start_time))
    print("Model score: %.3f" % gs.score(X_test, y_test))
    print("Best parameters: ", gs.best_params_)

In [6]:
Xcat_dataset = X_dataset.copy()
columns = ["Year", "Month", "Day", "Hour", "Minute", "Second", "DayOfWeek", "Browser Version", "Browser Minor"]
Xcat_dataset[columns] = Xcat_dataset[columns].astype("str")

In [7]:
cat_cols = Xcat_dataset.select_dtypes(include=["category","str"]).columns
num_cols = Xcat_dataset.select_dtypes(include=["int64","float64"]).columns
bool_cols = Xcat_dataset.select_dtypes(include=["bool"]).columns
passthrough_columns = [col for col in Xcat_dataset.columns if col not in cat_cols and col not in bool_cols and col not in num_cols]
    
numeric_transformer = Pipeline(
        steps = [("scaler", StandardScaler())]
    )

categorical_transformer = Pipeline([
        ("imputer", SimpleImputer(missing_values=np.nan, strategy="constant", fill_value="unknown")),
        ("encoder", OneHotEncoder(drop="first", handle_unknown="ignore"))
        ])
boolean_transformer = Pipeline([
        ("encoder", TargetEncoder(target_type="binary")),
        ])

preprocessor = ColumnTransformer(
        transformers=[
            ("cat", categorical_transformer, cat_cols),
            ("num", numeric_transformer, num_cols),
            ("bool", boolean_transformer, bool_cols)
        ]
    )

param_grid = {
    "classifier__n_estimators": [100, 200],
    "classifier__max_depth": [None, 10, 20],
    "classifier__min_samples_split": [2, 10, 30],
    "pca__n_components": [2, 6, 10, 15]
}

pipeline = Pipeline(
        steps=[
            ("preprocessor", preprocessor),
            ("pca", PCA(random_state=124)),
            ("classifier", RandomForestClassifier(random_state=124))
        ])
relaunch =""
while relaunch.lower() not in ["y", "n"]:
    relaunch = input("Do you want to relaunch the PCA analysis with more categorical columns? (y/n) ")
if relaunch.lower() == "y":
    X_train, X_test, y_train, y_test = train_test_split(Xcat_dataset, Y_true, test_size=0.2, stratify=Y_true, random_state=124)
    start_time = monotonic()
    gs = GridSearchCV(pipeline, param_grid, cv=5, n_jobs=2)
    gs.fit(X_train, y_train)
    print("Time taken to fit the model: %.2f seconds" % (monotonic() - start_time))
    print("Model score: %.3f" % gs.score(X_test, y_test))
    print("Best parameters: ", gs.best_params_)

In [11]:
Xcat_dataset = X_dataset.copy()
columns = ["Year", "Month", "Day", "Hour", "Minute", "Second", "DayOfWeek", "Browser Version", "Browser Minor",
           "Global Source IP", "Global Destination IP","IsWeekend", "Is Proxy", "Device Brand", "Device Model", "Browser Name","OS Name"]
Xcat_dataset = Xcat_dataset.drop(columns=columns)
bool_cols = Xcat_dataset.select_dtypes(include=["bool"]).columns
Xcat_dataset[bool_cols] = Xcat_dataset[bool_cols].astype("str")
cat_cols = Xcat_dataset.select_dtypes(include=["object","str"]).columns
num_cols = Xcat_dataset.select_dtypes(include="number").columns
# Xcat_dataset[cat_cols] = Xcat_dataset[cat_cols].apply(lambda x: x.fillna("unknown"))
# numeric_transformer = Pipeline(
#         steps = [
#             ("imputer", SimpleImputer(strategy="mean")),
#             ("scaler", StandardScaler())
#         ])
# Xcat_dataset[num_cols] = numeric_transformer.fit_transform(Xcat_dataset[num_cols])

X_train, X_test, y_train, y_test = train_test_split(Xcat_dataset, Y_true, test_size=0.2, stratify=Y_true, random_state=124)

In [12]:
passthrough_columns = [col for col in Xcat_dataset.columns if col not in cat_cols and col not in bool_cols and col not in num_cols]


numeric_transformer = Pipeline(
        steps = [
            ("imputer", SimpleImputer(strategy="mean")),
            ("scaler", StandardScaler())
        ])

cat_transformer = Pipeline([
        ("imputer", SimpleImputer(missing_values=np.nan, strategy="constant", fill_value="unknown")),
        ])

preprocessor = ColumnTransformer(
        transformers=[
            ("cat", cat_transformer, cat_cols),
            ("num", numeric_transformer, num_cols)
        ]).set_output(transform="pandas")

param_grid = {
    "classifier__n_estimators": [100, 200],
    "classifier__max_depth": [None, 10, 20],
    "classifier__min_samples_split": [2, 10, 30],
    "famd__n_components": [2, 6, 10, 15]
}

pipeline = Pipeline(
        steps=[
            ("preprocessor", preprocessor),
            ("famd", FAMD()),
            ("classifier", RandomForestClassifier(random_state=124))
        ])

In [13]:
relaunch =""
while relaunch.lower() not in ["y", "n"]:
    relaunch = input("Do you want to relaunch the FAMD analysis? (y/n) ")
if relaunch.lower() == "y":
    start_time = monotonic()
    gs = GridSearchCV(pipeline, param_grid, cv=5, n_jobs=2, pre_dispatch="n_jobs")
    gs.fit(X_train, y_train)
    print("Time taken to fit the model: %.2f seconds" % (monotonic() - start_time))
    print("Model score: %.3f" % gs.score(X_test, y_test))
    print("Best parameters: ", gs.best_params_)

/home/florians/cours/CyberSecurity_ML/.venv/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Time taken to fit the model: 1392.82 seconds
Model score: 0.328
Best parameters:  {'classifier__max_depth': 10, 'classifier__min_samples_split': 30, 'classifier__n_estimators': 100, 'famd__n_components': 10}


In [14]:
xtrain_processed = preprocessor.fit_transform(X_train)

In [19]:
famd = FAMD(n_components=10)
xtrain_famd = famd.fit_transform(xtrain_processed)
famd.eigenvalues_summary
famd.column_coordinates_

component,0,1,2,3,4,5,6,7,8,9
variable,,,,,,,,,,
num__Source Port,8.928017e-02,5.954733e-02,2.052499e-01,0.635559,1.002751e-02,0.000336,0.0,0.0,0.0,0.0
num__Destination Port,2.243330e-01,1.784741e-01,1.745877e-01,0.043482,6.571320e-02,0.313410,0.0,0.0,0.0,0.0
num__Packet Length,4.189435e-02,3.630124e-01,1.605216e-01,0.012424,3.390039e-01,0.083144,0.0,0.0,0.0,0.0
num__Anomaly Scores,6.424078e-02,3.027965e-01,1.427170e-01,0.050309,4.376600e-01,0.002277,0.0,0.0,0.0,0.0
num__Int Source IP,1.301309e-01,1.042106e-01,2.784212e-01,0.239791,1.284624e-01,0.118983,0.0,0.0,0.0,0.0
num__Int Destination IP,4.676763e-01,1.544636e-03,4.284583e-02,0.013912,9.331775e-03,0.464690,0.0,0.0,0.0,0.0
cat__Action Taken,7.599132e-06,7.710718e-05,1.860513e-05,0.000031,3.720570e-05,0.000090,NaN,NaN,NaN,NaN
cat__Alerts/Warnings,4.319694e-05,6.659830e-05,3.887245e-07,0.000070,2.099829e-07,0.000169,NaN,NaN,NaN,NaN
cat__Attack Signature,6.865104e-07,6.739979e-05,1.781571e-04,0.000023,8.286735e-05,0.000082,NaN,NaN,NaN,NaN


In [ ]:
Xcat_dataset = X_dataset.copy()
columns = ["Year", "Month", "Day", "Hour", "Minute", "Second", "DayOfWeek", "Browser Version", "Browser Minor"]
Xcat_dataset[columns] = Xcat_dataset[columns].astype("str")
bool_cols = Xcat_dataset.select_dtypes(include=["bool"]).columns
Xcat_dataset[bool_cols] = Xcat_dataset[bool_cols].astype("str")

In [ ]:
cat_cols = Xcat_dataset.select_dtypes(include=["category","str"]).columns
num_cols = Xcat_dataset.select_dtypes(include=["int64","float64"]).columns
passthrough_columns = [col for col in Xcat_dataset.columns if col not in cat_cols and col not in bool_cols and col not in num_cols]
    
numeric_transformer = Pipeline(
        steps = [
            ("imputer", SimpleImputer(strategy="mean")),
            ("scaler", StandardScaler())
        ])

cat_transformer = Pipeline([
        ("imputer", SimpleImputer(missing_values=np.nan, strategy="constant", fill_value="unknown")),
        ("encoder", OneHotEncoder(drop="first", handle_unknown="ignore", sparse_output=False))
        ])

preprocessor = ColumnTransformer(
        transformers=[
            ("cat", cat_transformer, cat_cols),
            ("num", numeric_transformer, num_cols)
        ])

param_grid = {
    "classifier__max_depth": [None, 10, 20],
    "classifier__min_samples_split": [2, 10, 30],
}

pipeline = Pipeline(
        steps=[
            ("preprocessor", preprocessor),
            ("classifier", RandomForestClassifier(random_state=124))
        ])


In [ ]:
relaunch =""
while relaunch.lower() not in ["y", "n"]:
    relaunch = input("Do you want to relaunch the HalvingGridSearchCV analysis? (y/n) ")
if relaunch.lower() == "y":
    X_train, X_test, y_train, y_test = train_test_split(Xcat_dataset, Y_true, test_size=0.2, stratify=Y_true, random_state=124)
    start_time = monotonic()
    gs = HalvingGridSearchCV(pipeline, param_grid, resource = "classifier__n_estimators", min_resources=10 , max_resources=500)
    gs.fit(X_train, y_train)
    print("Time taken to fit the model: %.2f seconds" % (monotonic() - start_time))
    print("Model score: %.3f" % gs.score(X_test, y_test))
    print("Best parameters: ", gs.best_params_)

In [ ]:
columns = ["Int Source IP", "Int Destination IP", "Source Port", "Destination Port", "Protocol", "Packet Type", "Traffic Type", "Attack Signature"]
Xsim_dataset = X_dataset[columns].copy()
bool_cols = Xsim_dataset.select_dtypes(include=["bool"]).columns
Xsim_dataset[bool_cols] = Xsim_dataset[bool_cols].astype("str")

In [ ]:
cat_cols = Xsim_dataset.select_dtypes(include=["category","str"]).columns
num_cols = Xsim_dataset.select_dtypes(include=["int64","float64"]).columns
passthrough_columns = [col for col in Xsim_dataset.columns if col not in cat_cols and col not in bool_cols and col not in num_cols]
    
numeric_transformer = Pipeline(
        steps = [
            ("imputer", SimpleImputer(strategy="mean")),
            ("scaler", StandardScaler())
        ])

cat_transformer = Pipeline([
        ("imputer", SimpleImputer(missing_values=np.nan, strategy="constant", fill_value="unknown")),
        ("encoder", OneHotEncoder(drop="first", handle_unknown="ignore", sparse_output=False))
        ])

preprocessor = ColumnTransformer(
        transformers=[
            ("cat", cat_transformer, cat_cols),
            ("num", numeric_transformer, num_cols)
        ])

param_grid = {
    "classifier__max_depth": [None, 10, 20],
    "classifier__min_samples_split": [2, 10, 30],
}

pipeline = Pipeline(
        steps=[
            ("preprocessor", preprocessor),
            ("classifier", RandomForestClassifier(random_state=124))
        ])


In [ ]:
relaunch =""
while relaunch.lower() not in ["y", "n"]:
    relaunch = input("Do you want to relaunch the simple HalvingGridSearchCV analysis? (y/n) ")
if relaunch.lower() == "y":
    X_train, X_test, y_train, y_test = train_test_split(Xsim_dataset, Y_true, test_size=0.2, stratify=Y_true, random_state=124)
    start_time = monotonic()
    gs = HalvingGridSearchCV(pipeline, param_grid, resource = "classifier__n_estimators", min_resources=10 , max_resources=1000, factor=2)
    gs.fit(X_train, y_train)
    print("Time taken to fit the model: %.2f seconds" % (monotonic() - start_time))
    print("Model score: %.3f" % gs.score(X_test, y_test))
    print("Best parameters: ", gs.best_params_)

In [ ]:
columns = ["Source Port", "Destination Port", "Protocol", "Packet Type", "Traffic Type", "Attack Signature", "Network Segment"]
Xsim_dataset = X_dataset[columns].copy()
bool_cols = Xsim_dataset.select_dtypes(include=["bool"]).columns
Xsim_dataset[bool_cols] = Xsim_dataset[bool_cols].astype("str")

In [ ]:
cat_cols = Xsim_dataset.select_dtypes(include=["category","str"]).columns
num_cols = Xsim_dataset.select_dtypes(include=["int64","float64"]).columns
passthrough_columns = [col for col in Xsim_dataset.columns if col not in cat_cols and col not in bool_cols and col not in num_cols]
    
numeric_transformer = Pipeline(
        steps = [
            ("imputer", SimpleImputer(strategy="mean")),
            ("scaler", StandardScaler())
        ])

cat_transformer = Pipeline([
        ("imputer", SimpleImputer(missing_values=np.nan, strategy="constant", fill_value="unknown")),
        ("encoder", OneHotEncoder(drop="first", handle_unknown="ignore", sparse_output=False))
        ])

preprocessor = ColumnTransformer(
        transformers=[
            ("cat", cat_transformer, cat_cols),
            ("num", numeric_transformer, num_cols)
        ])

param_grid = {
    "classifier__max_depth": [None, 10, 20],
    "classifier__min_samples_split": [2, 10, 30],
}

pipeline = Pipeline(
        steps=[
            ("preprocessor", preprocessor),
            ("classifier", RandomForestClassifier(random_state=124))
        ])


In [ ]:
relaunch =""
while relaunch.lower() not in ["y", "n"]:
    relaunch = input("Do you want to relaunch the simple HalvingGridSearchCV analysis? (y/n) ")
if relaunch.lower() == "y":
    X_train, X_test, y_train, y_test = train_test_split(Xsim_dataset, Y_true, test_size=0.2, stratify=Y_true, random_state=124)
    start_time = monotonic()
    gs = HalvingGridSearchCV(pipeline, param_grid, resource = "classifier__n_estimators", min_resources=10 , max_resources=500)
    gs.fit(X_train, y_train)
    print("Time taken to fit the model: %.2f seconds" % (monotonic() - start_time))
    print("Model score: %.3f" % gs.score(X_test, y_test))
    print("Best parameters: ", gs.best_params_)